In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [6]:
dep=dataset[['classification_yes']]

In [7]:
dataset['classification_yes'].value_counts()

classification_yes
True     249
False    150
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid={'n_neighbors':[5,10,50,100,500,1000],'algorithm':['auto','ball_tree','kd_tree','brute'],
            'weights':['uniform','distance'],'metric':['minkowski'],'p':[2,5,10]}
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [0.69838196 0.7214809  0.69048882 0.70573282 0.68591264 0.70557979
 0.71420667 0.7175126  0.71420667 0.71383728 0.71058198 0.70642609
 0.66149326 0.71207883 0.66149326 0.70118282 0.65727479 0.70118282
 0.63308515 0.68919019 0.64120663 0.68846998 0.61359754 0.68526283
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.69838196 0.7214809  0.69048882 0.70573282 0.68591264 0.70557979
 0.71420667 0.7175126  0.71420667 0.71383728 0.71058198 0.70642609
 0.66149326 0.71207883 0.66149326 0.70118282 0.65727479 0.70118282
 0.63308515 0.68919019 0.64120663 0.68846998 0.58831561 0.68526283
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.69838196 0.7214809  0.69048882 0.7057328

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'],
                         'n_neighbors': [5, 10, 50, 100, 500, 1000],
                         'p': [2, 5, 10], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [11]:
re=grid.cv_results_

grid_prediction=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

from sklearn.metrics import classification_report
clf_re=classification_report(y_test,grid_prediction)

C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NANDHAKUMAR GK\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~

In [12]:
print(cm)

[[46  5]
 [27 55]]


In [13]:
print(clf_re)

              precision    recall  f1-score   support

       False       0.63      0.90      0.74        51
        True       0.92      0.67      0.77        82

    accuracy                           0.76       133
   macro avg       0.77      0.79      0.76       133
weighted avg       0.81      0.76      0.76       133



In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_score value of the best parameter{}:".format(grid.best_params_),f1_macro)

The f1_score value of the best parameter{'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}: 0.7621040333682911


In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

np.float64(0.8297465327594451)